O módulo Python **requests** pode ser considerado o padrão *de fato* para o envio de solicitações HTTP, embora não faça parte das bibliotecas nativas da linguagem.

Se necessária a instalação, utilize o comando 

In [1]:
import requests
import re
from datetime import datetime
import time
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def criar_sessao_segura():
    """Cria uma sessão com retry e verificação SSL"""
    session = requests.Session()
    
    # Configura política de retry (3 tentativas com backoff)
    retry = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[500, 502, 503, 504]
    )
    
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    return session

def baixar_dados_transparencia(periodos, delay_segundos=2):
    base_url = "https://trescoracoes-mg.portaltp.com.br/api/transparencia.asmx/json_servidores"
    
    # Criar sessão segura
    session = criar_sessao_segura()
    
    for i, periodo in enumerate(periodos):
        try:
            # Parse da data no formato mm/aaaa
            data = datetime.strptime(periodo, '%m/%Y')
            mes = data.month
            ano = data.year
            
            # Baixar o arquivo XML com verificação SSL
            params = {'ano': ano, 'mes': mes}
            print(f"Baixando dados para {mes}/{ano}...")
            response = session.get(base_url, params=params, verify=True)  # verify=True é o padrão
            
            # Verificar se a requisição foi bem-sucedida
            response.raise_for_status()
            
            # Nome do arquivo no formato fptc_mmAANO.xml
            nome_arquivo_xml = f'fptc_{mes:02d}{ano}.xml'
            nome_arquivo_json = f'fptc_{mes:02d}{ano}.json'
            
            # Salvar o XML
            with open(nome_arquivo_xml, 'wb') as f:
                f.write(response.content)
            
            # Processar para remover tags e salvar como JSON
            with open(nome_arquivo_xml, 'r', encoding='utf-8') as xml_file:
                content = xml_file.read()
            
            clean_content = re.sub('<[^>]*>', '', content)
            
            with open(nome_arquivo_json, 'w', encoding='utf-8') as json_file:
                json_file.write(clean_content)

            # Remover o arquivo XML após processamento
            os.remove(nome_arquivo_xml)
                
            print(f"Arquivos para {mes:02d}/{ano} processados com sucesso!")
            
            # Adicionar delay entre requisições (exceto após o último item)
            if i < len(periodos) - 1:
                print(f"Aguardando {delay_segundos} segundos...")
                time.sleep(delay_segundos)
            
        except requests.exceptions.SSLError as e:
            print(f"Erro de SSL ao processar {periodo}: {str(e)}")
            print("Tentando com verificação SSL desativada (modo inseguro)...")
            # Fallback para verify=False se o SSL falhar
            try:
                response = session.get(base_url, params=params, verify=False)
                response.raise_for_status()
                # ... resto do processamento ...
            except Exception as fallback_e:
                print(f"Falha mesmo com verify=False: {str(fallback_e)}")
                
        except Exception as e:
            print(f"Erro ao processar {periodo}: {str(e)}")

# Lista de períodos a serem baixados
periodos = [#'11/2022', 
            #'12/2024', 
            #'06/2025',
            #'07/2025',
            '08/2025']

# Chamar a função com delay de 3 segundos entre requisições
baixar_dados_transparencia(periodos, delay_segundos=3)

Baixando dados para 8/2025...
✅ Arquivos para 08/2025 processados com sucesso!


In [11]:
import requests

# The API endpoint
url = "https://jsonplaceholder.typicode.com/posts/0"

# A GET request to the API
response = requests.get(url)

# Print the response
response_json = response.json()
print(response_json)


{'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}


In [3]:
# Print status code from original response (not JSON)
print(response.status_code)


200


In [4]:
# The API endpoint
url = "https://jsonplaceholder.typicode.com/posts/"

# Adding a payload
payload = {"id": [1, 2, 3], "userId":1}

# A get request to the API
response = requests.get(url, params=payload)

# Print the response
response_json = response.json()

for i in response_json:
    print(i, "\n")

{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'} 

{'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'} 

{'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'} 



In [6]:
# Define new data to create
new_data = {
    "userID": 1,
    "id": 1,
    "title": "Making a POST request",
    "body": "This is the data we created."
}

# The API endpoint to communicate with
url_post = "https://jsonplaceholder.typicode.com/posts"

# A POST request to tthe API
post_response = requests.post(url_post, json=new_data)

# Print the response
post_response_json = post_response.json()
print(post_response_json)

{'userID': 1, 'id': 101, 'title': 'Making a POST request', 'body': 'This is the data we created.'}


In [7]:
# Print status code from original response (not JSON)
print(post_response.status_code)

201


In [8]:
GET / HTTP/1.1
Host: developer.mozilla.org
Accept-Language: fr

SyntaxError: illegal target for annotation (685915712.py, line 3)